**This is a script for quickly demonstrating the capability of model Learning Möbius Transformations**

## 1. Import essential libraries.

In [1]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
import sympy as sp
import torch
import linecache
from pathlib import Path


## 2. Load model and Tokenizer

In [2]:
model_path = "./results/DomainReuctionModel" # model path to your trained model
tokenizer_path= "./results/DomainReuctionModel" # tokenizer path to your trained model (usually the same as model path)

In [3]:
tokenizer = T5Tokenizer.from_pretrained(tokenizer_path)
model = T5ForConditionalGeneration.from_pretrained(model_path)
print(f"max length: {model.config.n_positions}")

device='cuda'
model=model.to(device)
model=model.eval()
def generate_summary(input_tokens):
    inputs = tokenizer(input_tokens, return_tensors="pt",is_split_into_words=True, padding=True, truncation=True)
    if device =='cuda':
        inputs = {k: v.to('cuda') for k, v in inputs.items()}
        outputs = model.generate(inputs['input_ids'], max_length=16, num_beams=2, early_stopping=True)
    else:
        outputs = model.generate(inputs.input_ids, max_length=1024, num_beams=2, early_stopping=True)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


max length: 512


## 3. Use model to transforamtion

**You can use your own expression here, expression can be generated using the script in `./gen_data/different_methods.py`**

the input datapoint $z = \frac{p_{1}}{q_{1}} + i \frac{p_{2}}{q_{2}}$, use this form of input : [$p_1$,$p_2$,$q_1$,$q_2$]

the prediction of model is a transformation matrix $A = (k, l, m, n)$

In [44]:
test_input=['24033', '20000', '13287', '5000']
p1, q1, p2, q2 = test_input
z = sp.Rational(int(p1), int(q1)) + sp.I * sp.Rational(int(p2), int(q2))
print(f"test_input z = {z}")
generate_summary(test_input).split(" ")
k,l,m,n = generate_summary(test_input).split(" ")
print(f"model output transformation matrix: [[{k}, {l}], [{m}, {n}]]")

test_input z = 24033/20000 + 13287*I/5000
model output transformation matrix: [[1, -1], [0, 1]]


## 4. Check the prediction

check the prediction with modular transformation $z'=\frac{kz+l}{mz+n}$

In [45]:
def is_in_fundamental_domain(z):
    real_part = float(sp.re(z))
    modulus = float(sp.Abs(z))
    return (-0.5 <= real_part) & (real_part < 0.5) & (modulus >= 1)

In [46]:
transfered_z = (int(k) * z + int(l)) / (int(m) * z + int(n))
print(f"transfered z = {sp.sympify(transfered_z)}")
print(f"is in fundamental domain: {is_in_fundamental_domain(transfered_z)}")

transfered z = 4033/20000 + 13287*I/5000
is in fundamental domain: True
